In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-tts-deepfake-challenge/sample.csv


In [2]:
!pip install evaluate

import torch
from transformers import Trainer, TrainingArguments, AutoFeatureExtractor, AutoModelForAudioClassification
from datasets import load_dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import pandas as pd
import numpy as np
from scipy.special import softmax
from tqdm import tqdm

# Load dataset
dataset = load_dataset("SherryT997/IndicTTS-Deepfake-Challenge-Data")
train_dataset = dataset["train"].shuffle(seed=42)
test_dataset = dataset["test"]

# Load Model & Feature Extractor
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
model = AutoModelForAudioClassification.from_pretrained(model_id, num_labels=2)  # Binary classification

# Get unique languages
unique_languages = set(train_dataset["language"])

# Get 1/3rd of dataset while maintaining language balance
sampled_indices = []
fraction = 0.33

for lang in unique_languages:
    lang_indices = [i for i, lang_val in enumerate(train_dataset["language"]) if lang_val == lang]
    sample_size = max(1, int(len(lang_indices) * fraction))  # Ensure at least 1 sample per language
    sampled_indices.extend(lang_indices[:sample_size])

# Create new dataset with sampled indices
sampled_dataset = train_dataset.select(sampled_indices)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def prepare_sample(batch):
    audio = batch["audio"]["array"]
    expected_length = 32000  # 2 sec of audio (32,000 samples)

    # Trim or pad to 2 sec
    if len(audio) < expected_length:
        audio = np.pad(audio, (0, expected_length - len(audio)), mode='constant')
    else:
        audio = audio[:expected_length]

    # Extract features using feature_extractor
    inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    batch["input_values"] = inputs.input_values[0]  

    # Convert label to tensor
    batch["labels"] = torch.tensor(batch["is_tts"], dtype=torch.float)

    return batch

# Apply preprocessing
sampled_dataset = sampled_dataset.map(prepare_sample, remove_columns=["audio", "text", "id", "language", "is_tts"])

# Split dataset
dataset1 = sampled_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

from transformers import DataCollatorWithPadding
from evaluate import load

@dataclass
class DataCollatorWithPadding:
    processor: AutoFeatureExtractor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        return batch

data_collator = DataCollatorWithPadding(feature_extractor, padding=True)

# Load evaluation metrics
accuracy_metric = load("accuracy")
precision_metric = load("precision")
recall_metric = load("recall")
f1_metric = load("f1")
roc_auc_metric = load("roc_auc")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_probs = softmax(pred_logits, axis=-1)[:, 1]  # Probability of class 1 (synthetic speech)
    labels = pred.label_ids

    accuracy = accuracy_metric.compute(predictions=pred_probs.round(), references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["precision"]
    recall = recall_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["recall"]
    f1 = f1_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["f1"]
    
    roc_auc = roc_auc_metric.compute(prediction_scores=pred_probs, references=labels)["roc_auc"]


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc
    }


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


README.md:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

train-00000-of-00035.parquet:   0%|          | 0.00/453M [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10261 [00:00<?, ? examples/s]

In [3]:

# Freeze feature extractor
for param in model.hubert.parameters():  # Freeze the feature encoder
    param.requires_grad = False



training_args = TrainingArguments(
    output_dir="AsrTaskModel",
    group_by_length=True,
    per_device_train_batch_size=8,  # Increased batch size
    per_device_eval_batch_size=8,  # Keep evaluation batch size same
    evaluation_strategy="steps",
    metric_for_best_model="roc_auc",  # Optimizing for ROC-AUC instead of accuracy
    num_train_epochs=25,  # Set to 10 as per your preference
    fp16=True,  # Mixed precision training for speed
    gradient_checkpointing=True,  # Save memory
    save_steps=1000,
    eval_steps=500,
    logging_steps=500,
    learning_rate=3e-5,  # Lower LR to prevent instability
    weight_decay=0.01,  # Increased weight decay to reduce overfitting
    warmup_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    save_strategy="steps",
    report_to="none"
)



# Define trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset1["train"],
    eval_dataset=dataset1["test"],
    tokenizer=feature_extractor,
)

trainer.train()

# Inference & Submission


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-7d0153d76c52>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
500,0.690800,0.682522,0.665044,0.602136,0.907445,0.723917,0.808124
1000,0.661600,0.630097,0.774099,0.838875,0.659960,0.738739,0.847948
1500,0.585400,0.544489,0.788705,0.816742,0.726358,0.768903,0.860074
2000,0.505500,0.482551,0.806232,0.858173,0.718310,0.782037,0.882260
2500,0.449400,0.439026,0.819864,0.867925,0.740443,0.799131,0.898831
3000,0.407000,0.399171,0.839338,0.862445,0.794769,0.827225,0.913530
3500,0.375500,0.369547,0.852970,0.863445,0.826962,0.844810,0.924384
4000,0.352400,0.348274,0.864654,0.862348,0.857143,0.859738,0.931571
4500,0.333900,0.331893,0.867575,0.891540,0.826962,0.858038,0.938290
5000,0.330100,0.317289,0.872444,0.857422,0.883300,0.870168,0.942265


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-pac

TrainOutput(global_step=14450, training_loss=0.3395753178052011, metrics={'train_runtime': 6136.3644, 'train_samples_per_second': 37.62, 'train_steps_per_second': 2.355, 'total_flos': 1.0499876206272e+18, 'train_loss': 0.3395753178052011, 'epoch': 25.0})

In [4]:
model.eval()
submission_results = []

for sample in test_dataset:
    sample_id = sample["id"]
    audio_input = sample["audio"]["array"]

    inputs = feature_extractor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    probabilities = softmax(logits.cpu().numpy(), axis=-1)
    is_tts_prob = round(probabilities[0, 1], 3)

    submission_results.append([sample_id, is_tts_prob])

submission_df = pd.DataFrame(submission_results, columns=["id", "is_tts"])
submission_df.to_csv("./submission.csv", index=False)

print("✅ Submission file saved correctly with original 'id' values!")


✅ Submission file saved correctly with original 'id' values!
